# Converting Data to CSV

In [1]:
# Important libraries
import os
import wfdb
import numpy as np
import pandas as pd
from tqdm import tqdm

# File paths
fp_notebooks_folder = "./"
fp_code_folder = "../"
fp_raw_data_folder = os.path.join(fp_code_folder, "../mimic-database-1.0.0")
fp_processed_folder = os.path.join(fp_code_folder, "../processed_data")
fp_converted_folder = os.path.join(fp_code_folder, "../processed_data", "converted_to_csv")

## Signal Data

In [2]:
# Get Record Names
def get_record_names(fp_raw_data_folder, remove_trailing=2):
    fp_records_file = os.path.join(fp_raw_data_folder, "RECORDS")
    if not os.path.exists(fp_records_file):
        raise Exception("Records file does not exist!") 
    else:
        with open(fp_records_file, "r") as f:
            records = f.readlines()
            # remove trailing \n and slash
            records = [record[:-remove_trailing] for record in records]
        return records
    
records = get_record_names(fp_raw_data_folder)
print(len(records), "Records:", records)

72 Records: ['037', '039', '041', '055', '208', '209', '210', '211', '212', '213', '216', '218', '219', '220', '221', '222', '224', '225', '226', '230', '231', '237', '240', '248', '252', '253', '254', '260', '262', '276', '281', '284', '291', '401', '403', '404', '405', '408', '409', '410', '411', '413', '414', '415', '417', '418', '427', '430', '437', '438', '439', '442', '443', '444', '446', '449', '450', '451', '452', '453', '454', '456', '466', '471', '472', '474', '476', '477', '480', '482', '484', '485']


In [3]:
def filter_record_names(records, remove_trailing=None, filter_limit=55):
    # Only keep records after 055 (which contain alarms)
    # "al (annotations for alarms related to changes in the patient's status) and 
    # in (annotations related to changes in the functioning of the monitor) for all records after 055"
    if remove_trailing:
        return [record for record in records if int(record[:-remove_trailing])>=filter_limit]
    return [record for record in records if int(record)>=filter_limit]

filtered_records = filter_record_names(records)
print(len(filtered_records), "Filtered Records:", filtered_records)

69 Filtered Records: ['055', '208', '209', '210', '211', '212', '213', '216', '218', '219', '220', '221', '222', '224', '225', '226', '230', '231', '237', '240', '248', '252', '253', '254', '260', '262', '276', '281', '284', '291', '401', '403', '404', '405', '408', '409', '410', '411', '413', '414', '415', '417', '418', '427', '430', '437', '438', '439', '442', '443', '444', '446', '449', '450', '451', '452', '453', '454', '456', '466', '471', '472', '474', '476', '477', '480', '482', '484', '485']


In [4]:
# Get data in dataframe format for a single record's files
def get_record_data(fp_raw_data_folder, record_name):
    fp_record_folder = os.path.join(fp_raw_data_folder, record_name, record_name)
    
    # 1. Get signal data
    signal, fields = wfdb.rdsamp(fp_record_folder)
    # Check that signal length is the same as signal length in fields description
    assert signal.shape[0] == fields["sig_len"]
    # Check that we have units for all features
    colnames, colunits = fields['sig_name'], fields['units']
    assert len(colnames) == len(colunits)
    # Column names = feature (units)
    num_cols = len(colnames) 
    new_colnames = [colnames[i]+" ("+colunits[i]+")" for i in range(num_cols)]
    data_df = pd.DataFrame(signal, columns=new_colnames)
    
    # 2. Get annotations
    def add_annotation(ann_type, data_df, name):
        ann = wfdb.rdann(fp_record_folder, ann_type)
        ann_df = pd.DataFrame({"index": ann.sample, name: ann.aux_note})
        # Remove duplicate alarms
        ann_df = ann_df.drop_duplicates()
        fields[f"unique_{name}"] = str(ann_df[name].value_counts().to_dict())
        # If multiple alarms occur at the same time
        ann_df = ann_df.groupby('index').agg({name:'first'.join})
        # Add annotation to existing data_df
        data_df = data_df.join(ann_df, how='left')
        return data_df
    data_df = add_annotation("al", data_df, "alarms")
    data_df = add_annotation("abp", data_df, "abp")
    
    return data_df, fields

# Example of how a record data would appear
testing = get_record_data(fp_raw_data_folder, '409')
display(testing[0])
print(testing[1])

,II (mV),MCL1 (mV),V (mV),ABP (mmHg),PLETH (mV),RESP (mV),alarms,abp
0,-0.096797,-0.109935,-0.104863,101.803156,-0.0655,-0.24900,NaN,NaN
1,-0.131673,-0.114821,-0.121581,102.103681,-0.1240,-0.25025,NaN,NaN
2,-0.113167,-0.078990,-0.047112,102.554470,-0.1830,-0.25100,NaN,NaN
3,0.026335,0.009772,0.092705,103.080391,-0.2405,-0.25125,NaN,NaN
4,0.239858,0.126221,0.252280,103.681443,-0.2965,-0.25175,NaN,NaN
...,...,...,...,...,...,...,...,...
19498987,0.024199,0.009772,0.025836,117.580766,0.5330,NaN,NaN,NaN
19498988,0.009253,0.001629,0.010638,114.575507,0.4785,NaN,NaN,NaN
19498989,-0.022064,-0.020358,-0.004559,112.622089,0.4215,NaN,NaN,NaN
19498990,-0.046263,-0.061075,-0.047112,111.570248,0.3615,NaN,NaN,NaN


{'fs': 125, 'sig_len': 19498992, 'n_sig': 6, 'base_date': datetime.date(1995, 5, 5), 'base_time': datetime.time(11, 35, 23), 'units': ['mV', 'mV', 'mV', 'mmHg', 'mV', 'mV'], 'sig_name': ['II', 'MCL1', 'V', 'ABP', 'PLETH', 'RESP'], 'comments': [], 'unique_alarms': "{'ALARM: ** IRREGULAR HR   ': 83, 'ALARM: ** PAIR VPBs      ': 38, 'ALARM: ** MULTIFORM VPBs ': 38, 'ALARM: ** VPBs > 5/MIN   ': 8, 'ALARM: ** ABP  203 > 190 ': 5, 'ALARM: ** ABP  195 > 185 ': 5, 'ALARM: ** ABP  200 > 190 ': 5, 'ALARM: ** VPBs > 11/MIN  ': 5, 'ALARM: ** ABP  202 > 190 ': 4, 'ALARM: ** ABP  207 > 195 ': 4, 'ALARM: ** ABP  214 > 195 ': 3, 'ALARM: ** ABP  210 > 195 ': 3, 'ALARM: ** ABP  198 > 190 ': 3, 'ALARM: ** ABP  209 > 195 ': 3, 'ALARM: ** ABP  199 > 190 ': 3, 'ALARM: ** ABP  213 > 195 ': 3, 'ALARM: ** ABP  204 > 195 ': 2, 'ALARM: ** ABP  208 > 195 ': 2, 'ALARM: ** ABP  198 > 185 ': 2, 'ALARM: ** ABP  211 > 195 ': 2, 'ALARM: ** ABP  201 > 190 ': 2, 'ALARM: ** ABP    0 < 105 ': 2, 'ALARM: ** ABP  360 > 190 '

In [5]:
# NA values exist in columns
testing[0].isna().sum()

II (mV)           2466
MCL1 (mV)        12054
V (mV)              69
ABP (mmHg)           0
PLETH (mV)       66620
RESP (mV)      3438392
alarms        19498729
abp           19201757
dtype: int64

In [6]:
testing[0][testing[0]["alarms"].notna()].tail(50)

,II (mV),MCL1 (mV),V (mV),ABP (mmHg),PLETH (mV),RESP (mV),alarms,abp
13861916,0.081139,0.045603,0.018237,118.557476,-0.0875,0.51175,ALARM: ** ABP 208 > 195,NaN
13862424,0.041281,0.009772,0.003040,137.716003,0.3885,0.51175,ALARM: ** ABP 209 > 195,NaN
13876508,-0.011388,-0.020358,0.003040,147.858753,-0.1285,0.28600,ALARM: ** PAIR VPBs,NaN
13905952,0.145196,0.121336,0.075988,142.073629,-0.0070,-0.19225,ALARM: ** ABP 210 > 195,NaN
13906076,0.196441,0.157166,0.176292,203.756574,-0.1595,-0.29250,ALARM: ** PAIR VPBs,NaN
13906460,-0.015658,-0.002443,0.003040,127.798648,0.9835,NaN,ALARM: ** PAIR VPBs,NaN
13906464,-0.034164,-0.052117,-0.030395,122.314050,0.7670,NaN,ALARM: ** ABP 210 > 195,NaN
13907104,0.162278,0.126221,0.068389,118.707739,-0.1170,-0.34025,ALARM: ** ABP 213 > 195,NaN
13907740,-0.046263,-0.043160,-0.054711,102.253944,NaN,NaN,ALARM: ** PAIR VPBs,NaN
13907744,0.011388,-0.065147,-0.121581,110.368144,NaN,NaN,ALARM: ** ABP 213 > 195,NaN


In [7]:
fgap, bgap = 5, 5
index =13905952	
testing[0].iloc[index-fgap:index+bgap]

,II (mV),MCL1 (mV),V (mV),ABP (mmHg),PLETH (mV),RESP (mV),alarms,abp
13905947,0.106050,0.076547,0.003040,101.577761,0.0680,-0.14025,NaN,NaN
13905948,0.116726,0.090391,0.025836,102.028550,0.0545,-0.15375,NaN,221/101
13905949,0.123843,0.108306,0.034954,104.282494,0.0385,-0.16550,NaN,NaN
13905950,0.125267,0.108306,0.034954,110.894065,0.0250,-0.17575,NaN,NaN
13905951,0.131673,0.099349,0.042553,123.816679,0.0110,-0.18475,NaN,NaN
13905952,0.145196,0.121336,0.075988,142.073629,-0.0070,-0.19225,ALARM: ** ABP 210 > 195,NaN
13905953,0.162278,0.126221,0.101824,162.208866,-0.0205,-0.19775,NaN,NaN
13905954,0.166548,0.130293,0.151976,180.916604,-0.0320,-0.20175,NaN,NaN
13905955,0.165125,0.126221,0.168693,196.769346,-0.0390,-0.20425,NaN,NaN
13905956,0.153737,0.126221,0.168693,209.391435,-0.0345,-0.20575,NaN,NaN


## Periodic Measurements "Numerics"    fields_list = []


In [8]:
records_numerics = get_record_names(os.path.join(fp_raw_data_folder, "numerics"), 1)
print(len(records_numerics), "Records:", records_numerics)

121 Records: ['032n', '033n', '037n', '039n', '041n', '048n', '052n', '054n', '055n', '208n', '209n', '210n', '211n', '212n', '213n', '214n', '215n', '216n', '218n', '219n', '220n', '221n', '222n', '224n', '225n', '226n', '230n', '231n', '232n', '233n', '235n', '237n', '240n', '241n', '242n', '243n', '245n', '248n', '252n', '253n', '254n', '255n', '259n', '260n', '262n', '264n', '267n', '268n', '269n', '271n', '276n', '277n', '279n', '280n', '281n', '284n', '285n', '286n', '288n', '289n', '291n', '293n', '401n', '403n', '404n', '405n', '408n', '409n', '410n', '411n', '413n', '414n', '415n', '417n', '418n', '422n', '427n', '430n', '431n', '432n', '434n', '436n', '437n', '438n', '439n', '440n', '441n', '442n', '443n', '444n', '446n', '447n', '449n', '450n', '451n', '452n', '453n', '454n', '455n', '456n', '457n', '458n', '463n', '464n', '465n', '466n', '467n', '468n', '471n', '472n', '474n', '476n', '477n', '479n', '480n', '481n', '482n', '484n', '485n', '486n', '488n']


In [9]:
filtered_records_numerics = filter_record_names(records_numerics, 1, filter_limit=48)
print(len(filtered_records_numerics), "Filtered Records:", filtered_records_numerics)

116 Filtered Records: ['048n', '052n', '054n', '055n', '208n', '209n', '210n', '211n', '212n', '213n', '214n', '215n', '216n', '218n', '219n', '220n', '221n', '222n', '224n', '225n', '226n', '230n', '231n', '232n', '233n', '235n', '237n', '240n', '241n', '242n', '243n', '245n', '248n', '252n', '253n', '254n', '255n', '259n', '260n', '262n', '264n', '267n', '268n', '269n', '271n', '276n', '277n', '279n', '280n', '281n', '284n', '285n', '286n', '288n', '289n', '291n', '293n', '401n', '403n', '404n', '405n', '408n', '409n', '410n', '411n', '413n', '414n', '415n', '417n', '418n', '422n', '427n', '430n', '431n', '432n', '434n', '436n', '437n', '438n', '439n', '440n', '441n', '442n', '443n', '444n', '446n', '447n', '449n', '450n', '451n', '452n', '453n', '454n', '455n', '456n', '457n', '458n', '463n', '464n', '465n', '466n', '467n', '468n', '471n', '472n', '474n', '476n', '477n', '479n', '480n', '481n', '482n', '484n', '485n', '486n', '488n']


In [10]:
# Get data in dataframe format for a single record's files
def get_record_data_numeric(fp_raw_data_folder, record_name):
    fp_record_folder = os.path.join(fp_raw_data_folder, "numerics", record_name)
    
    # 1. Get signal data
    signal, fields = wfdb.rdsamp(fp_record_folder)
    # Check that signal length is the same as signal length in fields description
    assert signal.shape[0] == fields["sig_len"]
    # Check that we have units for all features
    colnames, colunits = fields['sig_name'], fields['units']
    assert len(colnames) == len(colunits)
    # Column names = feature (units)
    num_cols = len(colnames) 
    new_colnames = [colnames[i]+" ("+colunits[i]+")" for i in range(num_cols)]
    data_df = pd.DataFrame(signal, columns=new_colnames)
    
    # 2. Get annotations
    def add_annotation(ann_type, data_df, name):
        ann = wfdb.rdann(fp_record_folder, ann_type)
        ann_df = pd.DataFrame({"index": ann.sample, name: ann.aux_note})
        # Remove duplicate alarms
        ann_df = ann_df.drop_duplicates()
        fields[f"unique_{name}"] = str(ann_df[name].value_counts().to_dict())
        # If multiple alarms occur at the same time
        ann_df = ann_df.groupby('index').agg({name:'first'.join})
        # Add annotation to existing data_df
        data_df = data_df.join(ann_df, how='left')
        return data_df
    data_df = add_annotation("al", data_df, "alarms")
    
    return data_df, fields

testing2 = get_record_data_numeric(fp_raw_data_folder, "048n")
display(testing2[0])
print(testing2[1])

,ABPmean (mmHg),ABPsys (mmHg),ABPdias (mmHg),HR (bpm),PULSE (bpm),RESP (bpm),SpO2 (%),alarms
0,110.0,159.0,89.0,85.0,86.0,0.0,97.0,NaN
1,111.0,161.0,90.0,85.0,84.0,0.0,97.0,NaN
2,111.0,161.0,90.0,85.0,84.0,0.0,97.0,NaN
3,113.0,163.0,92.0,85.0,84.0,0.0,97.0,NaN
4,113.0,164.0,92.0,85.0,84.0,0.0,97.0,NaN
...,...,...,...,...,...,...,...,...
51133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


{'fs': 0.9765625, 'sig_len': 51138, 'n_sig': 7, 'base_date': datetime.date(1994, 12, 13), 'base_time': datetime.time(8, 38, 53), 'units': ['mmHg', 'mmHg', 'mmHg', 'bpm', 'bpm', 'bpm', '%'], 'sig_name': ['ABPmean', 'ABPsys', 'ABPdias', 'HR', 'PULSE', 'RESP', 'SpO2'], 'comments': [], 'unique_alarms': "{'*ASYSTOLE        ': 23, '*TACHY 216 > 180 ': 9, '*VENT TACHY      ': 8, 'RUN VPBs 3 - 9  ': 4, 'PAIR VPBs       ': 2, '*TACHY 215 > 180 ': 1, 'MISSED BEATS    ': 1}"}


In [11]:
testing2[0][testing2[0]["alarms"].notna()]

,ABPmean (mmHg),ABPsys (mmHg),ABPdias (mmHg),HR (bpm),PULSE (bpm),RESP (bpm),SpO2 (%),alarms
8342,NaN,NaN,NaN,0.0,79.0,11.0,100.0,PAIR VPBs
8343,NaN,NaN,NaN,0.0,79.0,10.0,100.0,PAIR VPBs
8344,NaN,NaN,NaN,215.0,79.0,9.0,100.0,RUN VPBs 3 - 9
8345,NaN,NaN,NaN,216.0,79.0,11.0,100.0,*TACHY 215 > 180
8346,NaN,NaN,NaN,216.0,79.0,11.0,100.0,*TACHY 216 > 180
8347,NaN,NaN,NaN,216.0,79.0,11.0,100.0,*TACHY 216 > 180
8348,NaN,NaN,NaN,216.0,78.0,11.0,100.0,*TACHY 216 > 180
8349,NaN,NaN,NaN,216.0,78.0,11.0,100.0,*TACHY 216 > 180
8350,NaN,NaN,NaN,216.0,78.0,11.0,100.0,*TACHY 216 > 180
8351,NaN,NaN,NaN,216.0,78.0,11.0,100.0,*TACHY 216 > 180


In [12]:
# NA values exist in columns
testing2[0].isna().sum()

ABPmean (mmHg)    45970
ABPsys (mmHg)     45970
ABPdias (mmHg)    45970
HR (bpm)          40928
PULSE (bpm)       40947
RESP (bpm)        40946
SpO2 (%)          40947
alarms            51090
dtype: int64

## Output Converted Record (Numerics)

In [13]:
def get_all_records(fp_raw_data_folder, fp_output, records):
    import csv
    if not os.path.exists(fp_output):
        os.makedirs(fp_output)
    fields_list = []
    pbar = tqdm(records)
    for record in pbar:
        pbar.set_description(f"Getting record data [{record}]...")
        # Output data_df
        fp_output_file = os.path.join(fp_output, record+".csv")
        if os.path.exists(fp_output_file):
            continue
        data_df, fields = get_record_data_numeric(fp_raw_data_folder, record)
        pbar.set_description(f"Outputing record data [{record}]...")
        data_df.to_csv(fp_output_file)
        # Append fields data to be outputted
        fields["record"] = record
        fields_list.append(fields)
    fp_output_file = os.path.join(fp_output, "fields.csv")
    fields_df = pd.DataFrame(fields_list)
    fields_df.to_csv(fp_output_file, quoting=csv.QUOTE_NONE, escapechar='/')
    print("All records converted to CSV!")
    
get_all_records(fp_raw_data_folder=fp_raw_data_folder, fp_output=fp_converted_folder, records=filtered_records_numerics)

Outputing record data [488n]...: 100%|█████████████████████████████████████████████████████████████████████████| 116/116 [01:52<00:00,  1.03it/s]

All records converted to CSV!
